## 针对RNN的培训调整
这个回调重新组合了一些调整，以正确训练RNN。 它们都来自Stephen Merity等人的这篇文章。

激活正则化：在权重衰减的基础上，我们应用另一种非常相似的正则化形式，其中包括增加损失的各个层的输出的所有平方（应用了丢失）的总和的缩放因子。RNN。 直观地，体重衰减试图让网络学习小重量，这是为了让模型学会产生较小的激活。

时间激活正则化：最后，我们将损失加上h_（t + 1） -  h_t的平方和的缩放因子，其中h_i是RNN的一层输出（在应用丢失之前） 时间步长i（句子的第i个字）。 这将鼓励模型产生在句子的两个连续单词之间不会变化太快的激活。

### class RNNTrainer
`RNNTrainer(learn:Learner, alpha:float=0.0, beta:float=0.0) :: LearnerCallback`

将lr调整重新组合为seq_len，AR和TAR的回调。

创建一个回调，为学习者添加RNN调整，以便使用bptt对数据进行培训。 alpha是AR的比例，beta是TAR的比例。

回调方法
你不要自己调用 - 他们会被fastai的Callback系统自动调用，以启用该类的功能。

<b>on_epoch_begin</b>

`on_epoch_begin(**kwargs)`

重置模型的隐藏状态。

<b>on_loss_begin</b>

`on_loss_begin(last_output:Tuple[Tensor, Tensor, Tensor], **kwargs)`

保存额外的输出以供稍后使用，仅返回真实输出。

fastai RNN返回last_output，它是三个元素的元组，真实输出（返回）和dropout之前和之后的隐藏状态（内部保存用于下一个函数）。

<b>on_backward_begin</b>

`on_backward_begin(last_loss:Rank0Tensor, last_input:Tensor, **kwargs)`

将AR和TAR应用于last_loss。